# Question 5

Here we will first read the JSON data ffrom the 'data.txt' file.
<br> <br>
As sum of weight of title and abstract is 1, the weight of other entries will be zero. So, we store title and abstract of each document in an array. Note that the order of documents doesnot change. That is, entry of title and abstract present at index i of array title and abstract will be refering to the (i+1)th document in the 'data.txt' file.
<br> <br>
After storing the data of title and abstract in two different arrays ('title' and 'abstract'), we will create functions which we will use while querying. The functions are:
<br>
- postingList: returns a posting list for a given term.
<br>
- termList: returns a term list for a given term.
<br>
- weightedZonePost: returns the score while using posting list.
<br>
- weightedZoneTerm: returns the temperary score while using posting list (we update this later in the code).
<br>
- ZoneScorePostTwo: returns Zone score while quering (posting list) for two terms.
<br>
- ZoneScorePostOne: returns Zone score while quering (posting list) for one term.
<br>
- ZoneScoreTermTwo: returns Zone score while quering (term list) for two terms.
<br>
- ZoneScoreTermOne: returns Zone score while quering (term list) for one term.
<br> <br>
In this approach, we will be creating the posting list after the query is requested. The other option was to create the posting lists for all possible terms (by iterating through the title and abstract), but this would consume lot of time and space.
<br>
We can also improve the current approach by storing the posting list of a term after a query for that term is made so that the stored posting list can be used when a query for that term is invoked for the second (and further) time, thus saving space and time. But we didn't implement this as it was out of the scope for the requirements of this question.
<br> <br>
*Note: We are converting the title and abstract to lowercase. Also we are looking for the substring in the title/abstract. For example: if we look for "schistosoma" and if "antischistosomal" is present in the abstract/title, then it will count it too.

In [133]:
import json
import json_lines
import time

In [22]:
# As weights of entries other than title and abstract is 0, we will be talking only about title and abstract
# includind pmid too for reference

with open('data.txt') as f:
    count = 0
    title = []
    abstract = []
    pmid = []
    for item in json_lines.reader(f):
        title.append(item['title'].lower())
        abstract.append(item['abstract'].lower())
        pmid.append(item['pmid'].lower())
        count += 1
    print(count)
    print(len(title))
    print(len(abstract))
    print(len(pmid))

# final count (number of lines) = 104879
# title array contains the title of all the documents in order (in lower case)
# abstract array contains the abstract of all the documents in order (in lower case)

104879
104879
104879
104879


In [3]:
# This creates a posting list for given term
# It returns a 2D array.
def postingList(term, title, abstract):
    p = []
    for i in range(len(title)):
        if term in title[i]:
            p.append([str(i+1) + ".title"])
            if term in abstract[i]:
                p[-1].append(str(i+1) + ".abstract")
        elif term in abstract[i]:
            p.append([str(i+1) + ".abstract"])
    return p

In [4]:
# This creates a term list for given term
# It returns a 2D array.
# at 0 index, it cotains title list and at 1 index, it cotains abstract list
def termList(term, title, abstract):
    t = [ [], []]
    for i in range(len(title)):
        if term in title[i]:
            t[0].append(i+1)
        if term in abstract[i]:
            t[1].append(i+1)
    return t

In [5]:
# Example output
post = postingList('monkey', title, abstract)
term = termList('monkey', title, abstract)

In [6]:
print(post)
print(" ")
print(term[0])
print(term[1])

[['17.title', '17.abstract'], ['80.abstract'], ['144.abstract'], ['167.abstract'], ['218.abstract'], ['311.title', '311.abstract'], ['357.abstract'], ['445.title', '445.abstract'], ['481.abstract'], ['485.abstract'], ['572.abstract'], ['679.abstract'], ['703.title', '703.abstract'], ['1083.title'], ['1709.abstract'], ['1898.abstract'], ['2167.abstract'], ['2225.abstract'], ['2545.title', '2545.abstract'], ['2638.abstract'], ['2701.abstract'], ['2985.abstract'], ['3025.title'], ['3820.abstract'], ['4410.abstract'], ['4495.abstract'], ['4694.abstract'], ['4971.abstract'], ['4998.abstract'], ['5068.title', '5068.abstract'], ['5339.abstract'], ['5414.abstract'], ['5592.title', '5592.abstract'], ['5744.title', '5744.abstract'], ['5936.abstract'], ['6020.abstract'], ['6132.abstract'], ['6349.abstract'], ['6434.title', '6434.abstract'], ['6779.abstract'], ['6922.abstract'], ['7238.abstract'], ['7684.abstract'], ['7889.title', '7889.abstract'], ['8220.abstract'], ['9379.abstract'], ['10155.tit

In [7]:
# Will return the score while using posting list

def weightedZonePost(p1, p2):
#     titleWeight = 0.7
#     abstractWeight = 0.3
    score = 0
    for list in [p1, p2]:
        for x in list:
            if 'title' in x:
                score += 0.7
            if 'abstract' in x:
                score += 0.3
    return score

In [8]:
# Example output
score = weightedZonePost(['17.title', '17.abstract'], ['17.title'])
print(score)

1.7


In [60]:
# Will return the score (temp) while using term list

def weightedZoneTerm(z1, z2):
#     titleWeight = 0.7
#     abstractWeight = 0.3
    score = 0
    if z1 == 't':
        score += 0.7
    elif z1 == 'a':
        score += 0.3
    if z2 == 't':
        score += 0.7
    elif z2 == 'a':
        score += 0.3
    
    return score

In [61]:
# Example output
score = weightedZoneTerm('t', 'a')
print(score)

1.0


In [32]:
# Returns Zone score while using posting list for two terms

def ZoneScorePostTwo(q1, q2, title, abstract):
    scores = []
    for _ in range(count):
        scores.append(0)
    
    postingQ1 = postingList(q1, title, abstract)
    postingQ2 = postingList(q2, title, abstract)
    
    i = 0
    j = 0
    while (i < len(postingQ1) and j < len(postingQ2)):
        p1 = postingQ1[i]
        p2 = postingQ2[j]
        
        docID1 = int(p1[0].split('.')[0])
        docID2 = int(p2[0].split('.')[0])
        
        if (docID1 == docID2):
            scores[docID1 - 1] = weightedZonePost(p1, p2)
            i += 1
            j += 1
        elif (docID1 < docID2):
            i += 1
        else:
            j += 1
    return scores

In [33]:
# Example output
scores = ZoneScorePostTwo("monkey", "development", title, abstract)
num1 = 0
print("Below is the list of document numbers, PMID and score for documents with non-zero scores.\n\n")
for i in range(len(scores)):
    if scores[i] > 0:
        num1 += 1
        print("Document Number:", i+1, "PMID:", pmid[i], "and Score:", scores[i])
print("\n\nTotal docs with non zero score: ", num1)

Below is the list of document numbers, PMID and score for documents with non-zero scores.


Document Number: 357 PMID: 16563157 and Score: 0.6
Document Number: 679 PMID: 3104022 and Score: 0.6
Document Number: 2545 PMID: 16595009 and Score: 1.3
Document Number: 3820 PMID: 18294402 and Score: 0.6
Document Number: 4410 PMID: 18237438 and Score: 0.6
Document Number: 4694 PMID: 18398475 and Score: 0.6
Document Number: 5068 PMID: 18523533 and Score: 2.0
Document Number: 5592 PMID: 18267012 and Score: 1.3
Document Number: 11300 PMID: 17054721 and Score: 0.6
Document Number: 12587 PMID: 19869874 and Score: 0.6
Document Number: 12624 PMID: 13525580 and Score: 0.6
Document Number: 12857 PMID: 19871591 and Score: 0.6
Document Number: 13236 PMID: 19870368 and Score: 0.6
Document Number: 13276 PMID: 19870827 and Score: 1.3
Document Number: 13304 PMID: 19870269 and Score: 1.3
Document Number: 13412 PMID: 13715352 and Score: 0.6
Document Number: 13440 PMID: 19870235 and Score: 1.3
Document Number: 1

In [37]:
# Returns Zone score while using posting list for one term

def ZoneScorePostOne(q1, title, abstract):
    scores = []
    for _ in range(count):
        scores.append(0)
    
    titleWeight = 0.7
    abstractWeight = 0.3
    postingQ1 = postingList(q1, title, abstract)
    
    for entry in postingQ1:
        tempScore = 0
        for i in entry:
            if 'title' in i:
                tempScore += 0.7
            if 'abstract' in i:
                tempScore += 0.3
        docID1 = int(entry[0].split('.')[0])
        scores[docID1 - 1] += tempScore
    return scores

In [39]:
# Example output
scores = ZoneScorePostOne("schistosoma", title, abstract)
num1 = 0
print("Below is the list of document numbers, PMID and score for documents with non-zero scores.\n\n")
for i in range(len(scores)):
    if scores[i] > 0:
        num1 += 1
        print("Document Number:", i+1, "PMID:", pmid[i], "and Score:", scores[i])
print("\n\nTotal docs with non zero score: ", num1)

Below is the list of document numbers, PMID and score for documents with non-zero scores.


Document Number: 755 PMID: 16789838 and Score: 1.0
Document Number: 3209 PMID: 16942390 and Score: 1.0
Document Number: 3876 PMID: 18366784 and Score: 1.0
Document Number: 4942 PMID: 18414649 and Score: 1.0
Document Number: 4944 PMID: 18294395 and Score: 1.0
Document Number: 7002 PMID: 18414646 and Score: 0.3
Document Number: 7712 PMID: 18373844 and Score: 0.3
Document Number: 8782 PMID: 17767713 and Score: 0.3
Document Number: 10175 PMID: 17727708 and Score: 1.0
Document Number: 11643 PMID: 74592107459210 and Score: 0.7
Document Number: 17348 PMID: 17411340 and Score: 0.7
Document Number: 20170 PMID: 17208299 and Score: 0.3
Document Number: 20756 PMID: 17157855 and Score: 1.0
Document Number: 21463 PMID: 17319953 and Score: 0.3
Document Number: 21507 PMID: 17081633 and Score: 1.0
Document Number: 22382 PMID: 18694485 and Score: 0.3
Document Number: 26095 PMID: 18698361 and Score: 1.0
Document N

In [129]:
# helping function to merge 2 sorted arrays
def mergeArrays(arr1, arr2, n1, n2):
    arr3 = [None] * (n1 + n2)
    i = 0
    j = 0
    k = 0
    
    while i < n1 and j < n2:
        if arr1[i] < arr2[j]:
            arr3[k] = str(arr1[i]) + '.t'
            k = k + 1
            i = i + 1
        else:
            arr3[k] = str(arr2[j]) + '.a'
            k = k + 1
            j = j + 1
    
    while i < n1:
        arr3[k] = str(arr1[i]) + '.t'
        k = k + 1
        i = i + 1
    
    while j < n2:
        arr3[k] = str(arr2[j]) + '.a'
        k = k + 1
        j = j + 1
    
    return arr3


# Returns Zone score while using term list for two terms
def ZoneScoreTermTwo(q1, q2, title, abstract):
    scores = []
    for _ in range(count):
        scores.append(0)
    
    termQ1 = termList(q1, title, abstract)
    termQ2 = termList(q2, title, abstract)
    
    # We do union of termQ1[0] and termQ1[1] (same for termQ2), then we will calculate score
    marray1 = mergeArrays(termQ1[0], termQ1[1], len(termQ1[0]), len(termQ1[1]))
    marray2 = mergeArrays(termQ2[0], termQ2[1], len(termQ2[0]), len(termQ2[1]))
        
    i = 0
    j = 0
    while (i < len(marray1) and j < len(marray2)):
        p1 = marray1[i]
        p2 = marray2[j]
        
        docID1 = int(p1.split('.')[0])
        z1 = p1.split('.')[1]
        docID2 = int(p2.split('.')[0])
        z2 = p2.split('.')[1]

        if (docID1 == docID2):
            temp = weightedZoneTerm(z1, z2)
            scores[docID1 - 1] += temp
            
# case when one q1 is present in title and abstract of doc1 and only in title, or abstract of doc2 ( or vice-versa)
            if (i+1 < len(marray1)):
                if (marray1[i+1] == str(docID1) + '.t'):
                    if (str(docID1) not in marray2[j+1]):
                        scores[docID1 - 1] += 0.7
            if (j+1 < len(marray2)):
                if (marray2[j+1] == str(docID2) + '.t'):
                    if (str(docID2) not in marray1[i+1]):
                        scores[docID1 - 1] += 0.7
            i += 1
            j += 1
        elif (docID1 < docID2):
            i += 1
        else:
            j += 1
    return scores

In [89]:
# Example output
scores = ZoneScoreTermTwo("monkey", "development", title, abstract)
num1 = 0

print("**Note: Please read 1.299999999 as 1.3 (this might be the issue with python while incrementing a value of variable)\n\n")

print("Below is the list of document numbers, PMID and score for documents with non-zero scores.\n\n")
for i in range(len(scores)):
    if scores[i] > 0:
        num1 += 1
        print("Document Number:", i+1, "PMID:", pmid[i], "and Score:", scores[i])
print("\n\nTotal docs with non zero score: ", num1)

**Note: Please read 1.299999999 as 1.3 (this might be the issue with python while incrementing a value of variable)


Below is the list of document numbers, PMID and score for documents with non-zero scores.


Document Number: 357 PMID: 16563157 and Score: 0.6
Document Number: 679 PMID: 3104022 and Score: 0.6
Document Number: 2545 PMID: 16595009 and Score: 1.2999999999999998
Document Number: 3820 PMID: 18294402 and Score: 0.6
Document Number: 4410 PMID: 18237438 and Score: 0.6
Document Number: 4694 PMID: 18398475 and Score: 0.6
Document Number: 5068 PMID: 18523533 and Score: 2.0
Document Number: 5592 PMID: 18267012 and Score: 1.2999999999999998
Document Number: 11300 PMID: 17054721 and Score: 0.6
Document Number: 12587 PMID: 19869874 and Score: 0.6
Document Number: 12624 PMID: 13525580 and Score: 0.6
Document Number: 12857 PMID: 19871591 and Score: 0.6
Document Number: 13236 PMID: 19870368 and Score: 0.6
Document Number: 13276 PMID: 19870827 and Score: 1.2999999999999998
Document Numbe

In [92]:
# Returns Zone score while using term list for one term

def ZoneScoreTermOne(q1, title, abstract):
    scores = []
    for _ in range(count):
        scores.append(0)
    
    titleWeight = 0.7
    abstractWeight = 0.3
    termQ1 = termList(q1, title, abstract)
    
    marray1 = mergeArrays(termQ1[0], termQ1[1], len(termQ1[0]), len(termQ1[1]))
    
    for entry in marray1:
        tempScore = 0
        if '.a' in entry:
            tempScore += 0.3
        if '.t' in entry:
            tempScore += 0.7
        docID1 = int(entry.split('.')[0])
        scores[docID1 - 1] += tempScore
    return scores

In [93]:
# Example output
scores = ZoneScoreTermOne("schistosoma", title, abstract)
num1 = 0
print("Below is the list of document numbers, PMID and score for documents with non-zero scores.\n\n")
for i in range(len(scores)):
    if scores[i] > 0:
        num1 += 1
        print("Document Number:", i+1, "PMID:", pmid[i], "and Score:", scores[i])
print("\n\nTotal docs with non zero score: ", num1)

Below is the list of document numbers, PMID and score for documents with non-zero scores.


Document Number: 755 PMID: 16789838 and Score: 1.0
Document Number: 3209 PMID: 16942390 and Score: 1.0
Document Number: 3876 PMID: 18366784 and Score: 1.0
Document Number: 4942 PMID: 18414649 and Score: 1.0
Document Number: 4944 PMID: 18294395 and Score: 1.0
Document Number: 7002 PMID: 18414646 and Score: 0.3
Document Number: 7712 PMID: 18373844 and Score: 0.3
Document Number: 8782 PMID: 17767713 and Score: 0.3
Document Number: 10175 PMID: 17727708 and Score: 1.0
Document Number: 11643 PMID: 74592107459210 and Score: 0.7
Document Number: 17348 PMID: 17411340 and Score: 0.7
Document Number: 20170 PMID: 17208299 and Score: 0.3
Document Number: 20756 PMID: 17157855 and Score: 1.0
Document Number: 21463 PMID: 17319953 and Score: 0.3
Document Number: 21507 PMID: 17081633 and Score: 1.0
Document Number: 22382 PMID: 18694485 and Score: 0.3
Document Number: 26095 PMID: 18698361 and Score: 1.0
Document N

In [96]:
# **************************************************************************************
# Now, as we are done with creating the functions, lets get the output for given queries.
# **************************************************************************************

In [114]:
# monkey and development
# in posting list encoding

q1 = "monkey"
q2 = "development"
scores = ZoneScorePostTwo(q1, q2, title, abstract)
num1 = 0
print("Below is the list of document numbers, PMID and score for documents with non-zero scores.")
print("The title and abstract is also printed after each entry.\n\n")
for i in range(len(scores)):
    if scores[i] > 0:
        num1 += 1
        print("Result number:", num1)
        print("Document Number:", i+1)
        print("PMID:", pmid[i])
        print("Score:", scores[i])
        print("Title:", title[i])
        print("Abstract:", abstract[i])
        print(" ")
print("\n\nTotal docs with non zero score: ", num1)

Below is the list of document numbers, PMID and score for documents with non-zero scores.
The title and abstract is also printed after each entry.


Result number: 1
Document Number: 357
PMID: 16563157
Score: 0.6
Title: onchocerca parasites and wolbachia endosymbionts: evaluation of a spectrum of antibiotic types for activity against onchocerca gutturosa in vitro
Abstract: background
the filarial parasites of major importance in humans contain the symbiotic bacterium wolbachia and recent studies have shown that targeting of these bacteria with antibiotics results in a reduction in worm viability, development, embryogenesis, and survival. doxycycline has been effective in human trials, but there is a need to develop drugs that can be given for shorter periods and to pregnant women and children. the world health organisation-approved assay to screen for anti-filarial activity in vitro uses male onchocerca gutturosa, with effects being determined by worm motility and viability as measured

In [115]:
print("Total number of docs (PMIDs) for above query:", num1)

Total number of docs (PMIDs) for above query: 90


In [116]:
# monkey and development
# in term list encoding

q1 = "monkey"
q2 = "development"
scores = ZoneScoreTermTwo(q1, q2, title, abstract)
num1 = 0
print("Below is the list of document numbers, PMID and score for documents with non-zero scores.")
print("The title and abstract is also printed after each entry.\n\n")
print("**Note: Please read 1.2999999 as 1.3 (this might be the issue with python while incrementing a value of variable)\n\n")
for i in range(len(scores)):
    if scores[i] > 0:
        num1 += 1
        print("Result number:", num1)
        print("Document Number:", i+1)
        print("PMID:", pmid[i])
        print("Score:", scores[i])
        print("Title:", title[i])
        print("Abstract:", abstract[i])
        print(" ")
print("\n\nTotal docs with non zero score: ", num1)

Below is the list of document numbers, PMID and score for documents with non-zero scores.
The title and abstract is also printed after each entry.


**Note: Please read 1.2999999 as 1.3 (this might be the issue with python while incrementing a value of variable)


Result number: 1
Document Number: 357
PMID: 16563157
Score: 0.6
Title: onchocerca parasites and wolbachia endosymbionts: evaluation of a spectrum of antibiotic types for activity against onchocerca gutturosa in vitro
Abstract: background
the filarial parasites of major importance in humans contain the symbiotic bacterium wolbachia and recent studies have shown that targeting of these bacteria with antibiotics results in a reduction in worm viability, development, embryogenesis, and survival. doxycycline has been effective in human trials, but there is a need to develop drugs that can be given for shorter periods and to pregnant women and children. the world health organisation-approved assay to screen for anti-filarial activi

In [117]:
print("Total number of docs (PMIDs) for above query:", num1)

Total number of docs (PMIDs) for above query: 90


In [118]:
# schistosoma
# in posting list encoding

q1 = "schistosoma"
scores = ZoneScorePostOne(q1, title, abstract)
num1 = 0
print("Below is the list of document numbers, PMID and score for documents with non-zero scores.")
print("The title and abstract is also printed after each entry.\n\n")
for i in range(len(scores)):
    if scores[i] > 0:
        num1 += 1
        print("Result number:", num1)
        print("Document Number:", i+1)
        print("PMID:", pmid[i])
        print("Score:", scores[i])
        print("Title:", title[i])
        print("Abstract:", abstract[i])
        print(" ")
print("\n\nTotal docs with non zero score: ", num1)

Below is the list of document numbers, PMID and score for documents with non-zero scores.
The title and abstract is also printed after each entry.


Result number: 1
Document Number: 755
PMID: 16789838
Score: 1.0
Title: schistosoma mansoni tgf-β receptor ii: role in host ligand-induced regulation of a schistosome target gene
Abstract: members of transforming growth factor-beta (tgf-β) superfamily play pivotal roles in development in multicellular organisms. we report the functional characterization of the schistosoma mansoni type ii receptor (smtβrii). mining of the s. mansoni expressed sequence tag (est) database identified an est clone that shows homology to the kinase domain of type ii receptors from different species. the amplified est sequence was used as a probe to isolate a cdna clone spanning the entire coding region of a type ii serine/threonine kinase receptor. the interaction of smtβrii with smtβri was elucidated and shown to be dependent on tgf-β ligand binding. furthermore

In [119]:
print("Total number of docs (PMIDs) for above query:", num1)

Total number of docs (PMIDs) for above query: 105


In [120]:
# schistosoma
# in term list encoding

q1 = "schistosoma"
scores = ZoneScoreTermOne(q1, title, abstract)
num1 = 0
print("Below is the list of document numbers, PMID and score for documents with non-zero scores.")
print("The title and abstract is also printed after each entry.\n\n")
print("**Note: Please read 1.2999999 as 1.3 (this might be the issue with python while incrementing a value of variable)\n\n")
for i in range(len(scores)):
    if scores[i] > 0:
        num1 += 1
        print("Result number:", num1)
        print("Document Number:", i+1)
        print("PMID:", pmid[i])
        print("Score:", scores[i])
        print("Title:", title[i])
        print("Abstract:", abstract[i])
        print(" ")
print("\n\nTotal docs with non zero score: ", num1)

Below is the list of document numbers, PMID and score for documents with non-zero scores.
The title and abstract is also printed after each entry.


**Note: Please read 1.2999999 as 1.3 (this might be the issue with python while incrementing a value of variable)


Result number: 1
Document Number: 755
PMID: 16789838
Score: 1.0
Title: schistosoma mansoni tgf-β receptor ii: role in host ligand-induced regulation of a schistosome target gene
Abstract: members of transforming growth factor-beta (tgf-β) superfamily play pivotal roles in development in multicellular organisms. we report the functional characterization of the schistosoma mansoni type ii receptor (smtβrii). mining of the s. mansoni expressed sequence tag (est) database identified an est clone that shows homology to the kinase domain of type ii receptors from different species. the amplified est sequence was used as a probe to isolate a cdna clone spanning the entire coding region of a type ii serine/threonine kinase receptor. 

In [121]:
print("Total number of docs (PMIDs) for above query:", num1)

Total number of docs (PMIDs) for above query: 105


In [122]:
# cholera and risk
# in posting list encoding

q1 = "cholera"
q2 = "risk"
scores = ZoneScorePostTwo(q1, q2, title, abstract)
num1 = 0
print("Below is the list of document numbers, PMID and score for documents with non-zero scores.")
print("The title and abstract is also printed after each entry.\n\n")
for i in range(len(scores)):
    if scores[i] > 0:
        num1 += 1
        print("Result number:", num1)
        print("Document Number:", i+1)
        print("PMID:", pmid[i])
        print("Score:", scores[i])
        print("Title:", title[i])
        print("Abstract:", abstract[i])
        print(" ")
print("\n\nTotal docs with non zero score: ", num1)

Below is the list of document numbers, PMID and score for documents with non-zero scores.
The title and abstract is also printed after each entry.


Result number: 1
Document Number: 27010
PMID: 19445712
Score: 1.3
Title: effectiveness and economic analysis of the whole cell/recombinant b subunit (wc/rbs) inactivated oral cholera vaccine in the prevention of traveller's diarrhoea
Abstract: background
nowadays there is a debate about the indication of the oral whole-cell/recombinant b-subunit cholera vaccine (wc/rbs) in traveller's diarrhoea. however, a cost-benefit analysis based on real data has not been published.
methods
a cost-effectiveness and cost-benefit study of the oral cholera vaccine (wc/rbs), dukoral® for the prevention of traveller's diarrhoea (td) was performed in subjects travelling to cholera risk areas. the effectiveness of wc/rbs vaccine in the prevention of td was analyzed in 362 travellers attending two international vaccination centres in spain between may and sept

In [123]:
print("Total number of docs (PMIDs) for above query:", num1)

Total number of docs (PMIDs) for above query: 10


In [124]:
# cholera and risk
# in term list encoding

q1 = "cholera"
q2 = "risk"
scores = ZoneScoreTermTwo(q1, q2, title, abstract)
num1 = 0
print("Below is the list of document numbers, PMID and score for documents with non-zero scores.")
print("The title and abstract is also printed after each entry.\n\n")
print("**Note: Please read 1.2999999 as 1.3 (this might be the issue with python while incrementing a value of variable)\n\n")
for i in range(len(scores)):
    if scores[i] > 0:
        num1 += 1
        print("Result number:", num1)
        print("Document Number:", i+1)
        print("PMID:", pmid[i])
        print("Score:", scores[i])
        print("Title:", title[i])
        print("Abstract:", abstract[i])
        print(" ")
print("\n\nTotal docs with non zero score: ", num1)

Below is the list of document numbers, PMID and score for documents with non-zero scores.
The title and abstract is also printed after each entry.


**Note: Please read 1.2999999 as 1.3 (this might be the issue with python while incrementing a value of variable)


Result number: 1
Document Number: 27010
PMID: 19445712
Score: 1.2999999999999998
Title: effectiveness and economic analysis of the whole cell/recombinant b subunit (wc/rbs) inactivated oral cholera vaccine in the prevention of traveller's diarrhoea
Abstract: background
nowadays there is a debate about the indication of the oral whole-cell/recombinant b-subunit cholera vaccine (wc/rbs) in traveller's diarrhoea. however, a cost-benefit analysis based on real data has not been published.
methods
a cost-effectiveness and cost-benefit study of the oral cholera vaccine (wc/rbs), dukoral® for the prevention of traveller's diarrhoea (td) was performed in subjects travelling to cholera risk areas. the effectiveness of wc/rbs vaccine i

In [125]:
print("Total number of docs (PMIDs) for above query:", num1)

Total number of docs (PMIDs) for above query: 10


In [126]:
# mosquito and filariasis
# in posting list encoding

q1 = "mosquito"
q2 = "filariasis"
scores = ZoneScorePostTwo(q1, q2, title, abstract)
num1 = 0
print("Below is the list of document numbers, PMID and score for documents with non-zero scores.")
print("The title and abstract is also printed after each entry.\n\n")
for i in range(len(scores)):
    if scores[i] > 0:
        num1 += 1
        print("Result number:", num1)
        print("Document Number:", i+1)
        print("PMID:", pmid[i])
        print("Score:", scores[i])
        print("Title:", title[i])
        print("Abstract:", abstract[i])
        print(" ")
print("\n\nTotal docs with non zero score: ", num1)

Below is the list of document numbers, PMID and score for documents with non-zero scores.
The title and abstract is also printed after each entry.


Result number: 1
Document Number: 1028
PMID: 16723020
Score: 0.6
Title: concomitant infections of plasmodium falciparum and wuchereria bancrofti on the kenyan coast
Abstract: background
anopheles gambiae s.l. and an. funestus are important vectors of malaria and bancroftian filariasis, which occur as co-endemic infections along the kenyan coast. however, little is known about the occurrence and prevalence of concomitant infections of the two diseases in mosquito and human populations in these areas. this study reports the prevalence of concomitant infections of plasmodium falciparum and wuchereria bancrofti in mosquito and human populations in jilore and shakahola villages in malindi, kenya.
methods
mosquitoes were sampled inside houses by pyrethrum spray sheet collection (psc) while blood samples were collected by finger prick technique a

In [127]:
print("Total number of docs (PMIDs) for above query:", num1)

Total number of docs (PMIDs) for above query: 13


In [130]:
# mosquito and filariasis
# in term list encoding

q1 = "mosquito"
q2 = "filariasis"
scores = ZoneScoreTermTwo(q1, q2, title, abstract)
num1 = 0
print("Below is the list of document numbers, PMID and score for documents with non-zero scores.")
print("The title and abstract is also printed after each entry.\n\n")
print("**Note: Please read 1.2999999 as 1.3 (this might be the issue with python while incrementing a value of variable)\n\n")
for i in range(len(scores)):
    if scores[i] > 0:
        num1 += 1
        print("Result number:", num1)
        print("Document Number:", i+1)
        print("PMID:", pmid[i])
        print("Score:", scores[i])
        print("Title:", title[i])
        print("Abstract:", abstract[i])
        print(" ")
print("\n\nTotal docs with non zero score: ", num1)

Below is the list of document numbers, PMID and score for documents with non-zero scores.
The title and abstract is also printed after each entry.


**Note: Please read 1.2999999 as 1.3 (this might be the issue with python while incrementing a value of variable)


Result number: 1
Document Number: 1028
PMID: 16723020
Score: 0.6
Title: concomitant infections of plasmodium falciparum and wuchereria bancrofti on the kenyan coast
Abstract: background
anopheles gambiae s.l. and an. funestus are important vectors of malaria and bancroftian filariasis, which occur as co-endemic infections along the kenyan coast. however, little is known about the occurrence and prevalence of concomitant infections of the two diseases in mosquito and human populations in these areas. this study reports the prevalence of concomitant infections of plasmodium falciparum and wuchereria bancrofti in mosquito and human populations in jilore and shakahola villages in malindi, kenya.
methods
mosquitoes were sampled in

In [131]:
print("Total number of docs (PMIDs) for above query:", num1)

Total number of docs (PMIDs) for above query: 13


In [132]:
# **************************************************************************************
# Now, lets execute all these 8 queries 100 times and get the time taken to execute them.
# **************************************************************************************

In [150]:
# monkey and development
# in posting list encoding

start = time.time()
for _ in range(100):
    q1 = "monkey"
    q2 = "development"
    scores = ZoneScorePostTwo(q1, q2, title, abstract)
end = time.time()
time1 = end - start
print("It took %.8f seconds." % time1)

It took 41.49657989 seconds.


In [151]:
# monkey and development
# in term list encoding

start = time.time()
for _ in range(100):
    q1 = "monkey"
    q2 = "development"
    scores = ZoneScoreTermTwo(q1, q2, title, abstract)
end = time.time()
time2 = end - start
print("took %.8f seconds." % time2)

took 37.17105484 seconds.


In [152]:
# schistosoma
# in posting list encoding

start = time.time()
for _ in range(100):
    q1 = "schistosoma"
    scores = ZoneScorePostOne(q1, title, abstract)
end = time.time()
time3 = end - start
print("took %.8f seconds." % time3)

took 21.86803842 seconds.


In [153]:
# schistosoma
# in term list encoding

start = time.time()
for _ in range(100):
    q1 = "schistosoma"
    scores = ZoneScoreTermOne(q1, title, abstract)
end = time.time()
time4 = end - start
print("took %.8f seconds." % time4)

took 18.65261769 seconds.


In [154]:
# cholera and risk
# in term posting encoding

start = time.time()
for _ in range(100):
    q1 = "cholera"
    q2 = "risk"
    scores = ZoneScorePostTwo(q1, q2, title, abstract)
end = time.time()
time5 = end - start
print("took %.8f seconds." % time5)

took 48.20676684 seconds.


In [155]:
# cholera and risk
# in term list encoding

start = time.time()
for _ in range(100):
    q1 = "cholera"
    q2 = "risk"
    scores = ZoneScoreTermTwo(q1, q2, title, abstract)
end = time.time()
time6 = end - start
print("took %.8f seconds." % time6)

took 46.07700634 seconds.


In [159]:
# mosquito and filariasis
# in posting list encoding

start = time.time()
for _ in range(100):
    q1 = "mosquito"
    q2 = "filariasis"
    scores = ZoneScorePostTwo(q1, q2, title, abstract)
end = time.time()
time7 = end - start
print("took %.8f seconds." % time7)

took 39.15217447 seconds.


In [160]:
# mosquito and filariasis
# in term list encoding

start = time.time()
for _ in range(100):
    q1 = "mosquito"
    q2 = "filariasis"
    scores = ZoneScoreTermTwo(q1, q2, title, abstract)
end = time.time()
time8 = end - start
print("took %.8f seconds." % time8)

took 39.16410398 seconds.


In [161]:
# **************************************************************************************
# Done, lets look at the results
# **************************************************************************************

As we can see, for a given query: time taken to execute is less while using term list (dictionary) than while using posting list.
<br> <br>
This which is true theoritically.
<br> <br>
Reason: Searchig in dictionary is O(1) and to get a particular element in posting list, we need to tracerse through all the elements present before it. Hence less time is required in term (dictionary) indexing.
<br> <br>
(Note: For the last query, time in both the cases is almost the same.)

In [162]:
# **************************************************************************************
# Thank you
# **************************************************************************************